In [21]:
# %load_ext dotenv
# %dotenv

from dotenv import load_dotenv
from openai import OpenAI
import os
load_dotenv(override=True)

True

# non-v1 format

In [25]:

load_dotenv()
from openai import AzureOpenAI

api_key = os.getenv('AZURE_API_KEY')
if api_key:
    print('api key: ', api_key[:3] + '...' + api_key[-5:])
else:
    print('api key: None')

client = AzureOpenAI(
    api_version='2024-05-01-preview',
    api_key=os.getenv('AZURE_API_KEY'),
    azure_endpoint='https://ideta-gpt4.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview'
)

completion = client.chat.completions.create(
    model='azure',
    messages=[
        {"role": "system", "content": "xxx"},
        {
            "content": "I request to know about the pricing of your company?",
            "role": "user",
        },
        {
            "role": "assistant",
            "tool_calls": [
                {
                    "function": {
                        "arguments": '{"request":"What is the pricing of your company?"}',
                        "name": "NmesgxEEXZ-71sx4cqE",
                    },
                    "id": "call_K695KTX4cMVLxeNcIXRVl9qv",
                    "type": "function",
                },
            ],
        },
        {
            "role": "tool",
            "tool_call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
            "content": "Some data was displayed to the user",
        },
        {"role": "user", "content": "What is the pricing of your company?"},
    ],
    stream=True
)

infos = {}

for chunk in completion:
    if chunk.choices:
        content = chunk.choices[0].delta.content or ""
        print(content, end="")
    else:
        completion_tokens = getattr(chunk.usage, 'completion_tokens', infos.get('completionTokens', 0))
        prompt_tokens = getattr(chunk.usage, 'prompt_tokens', infos.get('promptTokens', 0))
        infos = {
            "completionTokens": completion_tokens,
            "promptTokens": prompt_tokens
        }
        print(f"infos: {infos}")

api key:  8ea...1bab0
infos: {'completionTokens': 0, 'promptTokens': 0}
I'm sorry for any confusion, but I don't have information regarding specific company pricing. Pricing often varies based on products, services, and other factors. It would be best to visit the company's official website or contact their customer service directly for the most accurate and up-to-date pricing information.

# v1 format

In [40]:
# response model (no stream)
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('AZURE_V1_API_KEY'),
    # full: https://bymycarai.openai.azure.com/openai/v1/responses
    base_url="https://bymycarai.openai.azure.com/openai/v1"
)

response = client.responses.create(
    model=os.getenv('AZURE_V1_DEPLOYMENT_NAME'),  # Replace with your model deployment name
    #   input="This is a test."
    # input=[
    #     {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
    #     {"role": "user", "content": "Who were the founders of Microsoft?"}
    # ]
    input=[
        {"role": "system", "content": "xxx"},
        {
            "content": "I request to know about the pricing of your company?",
            "role": "user",
        },
        # {
        #     "role": "assistant",
        #     "tool_calls": [
        #         {
        #             "function": {
        #                 "arguments": '{"request":"What is the pricing of your company?"}',
        #                 "name": "NmesgxEEXZ-71sx4cqE",
        #             },
        #             "id": "call_K695KTX4cMVLxeNcIXRVl9qv",
        #             "type": "function",
        #         },
        #     ],
        # },
        {
            "type": "function_call",
            "arguments": "{\"location\":\"San Francisco\"}",
            "call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
            "name": "NmesgxEEXZ-71sx4cqE"
        },
        # {
        #     "role": "tool",
        #     "tool_call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
        #     "content": "Some data was displayed to the user",
        # },
        {
            "type": "function_call_output",
            "call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
            "output": 'Some data was displayed to the user',
        },
        {"role": "user", "content": "What is the pricing of your company?"},
    ]
)

print(f"response.output[0].content[0].text: {response.output[0].content[0].text}")
print(f"response.usage.input_tokens: {response.usage.input_tokens}")
print(f"response.usage.output_tokens: {response.usage.output_tokens}")
# print(response.model_dump_json(indent=2))

response.output[0].content[0].text: It looks like I don't have information about a specific company attached to your question. Could you please clarify which company's pricing details you are interested in? If you are referring to OpenAI's API pricing or another service, let me know so I can provide accurate information for you!
response.usage.input_tokens: 83
response.usage.output_tokens: 56


In [ ]:
# response model (with stream)
# https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-key#streaming
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('AZURE_V1_API_KEY'),
    # full: https://bymycarai.openai.azure.com/openai/v1/responses
    base_url="https://bymycarai.openai.azure.com/openai/v1"
)

response = client.responses.create(
    model=os.getenv('AZURE_V1_DEPLOYMENT_NAME'),  # Replace with your model deployment name
    #   input="This is a test.",
    input=[
        {"role": "system", "content": "xxx"},
        {
            "content": "I request to know about the pricing of your company?",
            "role": "user",
        },
        {
            "role": "assistant",
            "tool_calls": [
                {
                    "function": {
                        "arguments": '{"request":"What is the pricing of your company?"}',
                        "name": "NmesgxEEXZ-71sx4cqE",
                    },
                    "id": "call_K695KTX4cMVLxeNcIXRVl9qv",
                    "type": "function",
                },
            ],
        },
        {
            "role": "tool",
            "tool_call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
            "content": "Some data was displayed to the user",
        },
        {"role": "user", "content": "What is the pricing of your company?"},
    ],
    stream=True
)

for event in response:
    # print(f"event: {event}")
    if event.type == 'response.output_text.delta':
        print(event.delta, end='')
        # print(f"event.delta: {event.delta}")
    if event.type == 'response.completed':
        print(event.response.usage)

event.delta: It
event.delta:  looks
event.delta:  like
event.delta:  you
event.delta: ’re
event.delta:  asking
event.delta:  about
event.delta:  company
event.delta:  pricing
event.delta: ,
event.delta:  but
event.delta:  you
event.delta:  haven't
event.delta:  specified
event.delta:  which
event.delta:  company
event.delta:  you
event.delta: ’re
event.delta:  referring
event.delta:  to
event.delta: .
event.delta:  Can
event.delta:  you
event.delta:  please
event.delta:  clarify
event.delta:  the
event.delta:  name
event.delta:  of
event.delta:  the
event.delta:  company
event.delta:  or
event.delta:  the
event.delta:  specific
event.delta:  product
event.delta: /service
event.delta:  you
event.delta:  are
event.delta:  interested
event.delta:  in
event.delta: ?
event.delta:  That
event.delta:  way
event.delta: ,
event.delta:  I
event.delta:  can
event.delta:  provide
event.delta:  accurate
event.delta:  and
event.delta:  relevant
event.delta:  information
event.delta:  about
event.del

# Reponses API function calling

https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-key#function-calling

In [38]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="https://bymycarai.openai.azure.com/openai/v1/",
    api_key=os.getenv("AZURE_V1_API_KEY")
)

api_key = os.getenv('AZURE_V1_API_KEY')
if api_key:
    print('api key: ', api_key[:3] + '...' + api_key[-5:])
else:
    print('api key: None')

print(f"deployment_name: {os.getenv('AZURE_V1_DEPLOYMENT_NAME')}")

response = client.responses.create(
    model=os.getenv('AZURE_V1_DEPLOYMENT_NAME'),  # replace with your model deployment name
    tools=[
        {
            "type": "function",
            "name": "get_weather",
            "description": "Get the weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                },
                "required": ["location"],
            },
        }
    ],
    input=[{"role": "user", "content": "What's the weather in San Francisco?"}],
)

print(response.model_dump_json(indent=2))

# # To provide output to tools, add a response for each tool call to an array passed
# # to the next response as `input`
# input = []
# for output in response.output:
#     if output.type == "function_call":
#         if output.name == "get_weather":
#             input.append(
#                 {
#                     "type": "function_call_output",
#                     "call_id": output.call_id,
#                     "output": '{"temperature": "70 degrees"}',
#                 }
#             )
#         else:
#             raise ValueError(f"Unknown function call: {output.name}")

# second_response = client.responses.create(
#     model=os.getenv('AZURE_V1_DEPLOYMENT_NAME'),
#     previous_response_id=response.id,
#     input=input
# )

# print(second_response.model_dump_json(indent=2))

api key:  EsP...G1LQj
deployment_name: gpt-4.1
{
  "id": "resp_0369ee087f8ed3ba00690ccf9bf3dc819080ba61b11e0113be",
  "created_at": 1762447259.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4.1",
  "object": "response",
  "output": [
    {
      "arguments": "{\"location\":\"San Francisco\"}",
      "call_id": "call_1i7yQeKLZn3vjdOyRoopuxUI",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_0369ee087f8ed3ba00690ccf9c91788190bad702b119e18f80",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string"
          }
        },
        "required": [
          "location"
        ],
        "additionalProperties": false
      },
      "strict": true,
      "type": "function",
   

In [39]:
# MULTIPLE FUNCTION CALLS

import os
from openai import OpenAI

client = OpenAI(
    base_url="https://bymycarai.openai.azure.com/openai/v1/",
    api_key=os.getenv("AZURE_V1_API_KEY")
)

response = client.responses.create(
    model=os.getenv('AZURE_V1_DEPLOYMENT_NAME'),
    tools=[
        {
            "type": "function",
            "name": "get_weather",
            "description": "Get the weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                },
                "required": ["location"],
            },
        },
        {
            "type": "function",
            "name": "get_time",
            "description": "Get the current time for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "timezone": {"type": "string"},
                },
                "required": ["location"],
            },
        }
    ],
    input=[
        {
            "role": "user", 
            "content": "What's the weather and current time in San Francisco?"
        }
    ],
)

print(response.model_dump_json(indent=2))

{
  "id": "resp_0fee6af0d556162800690ce5aa8d68819696af6ddcb9511a6e",
  "created_at": 1762452906.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4.1",
  "object": "response",
  "output": [
    {
      "arguments": "{\"location\":\"San Francisco\"}",
      "call_id": "call_KZh3LSgtfY9zqq1M4C1UYazs",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_0fee6af0d556162800690ce5abdfb08196839674553ba16551",
      "status": "completed"
    },
    {
      "arguments": "{\"location\":\"San Francisco\",\"timezone\":\"America/Los_Angeles\"}",
      "call_id": "call_GGfVAWU2f0F2rvrqVfoXj8ps",
      "name": "get_time",
      "type": "function_call",
      "id": "fc_0fee6af0d556162800690ce5ac15148196a2d47e1ab35d3efc",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": 

# Convert chat completion messages to responses API messages

In [42]:
import json


def convert_chat_completion_to_response_format(messages: list[dict]) -> list[dict]:
    """
    Convert messages from Chat Completion API format to Response API format

    Transforms OpenAI Chat Completion format (with tool_calls and tool roles)
    to Response API format (with function_call and function_call_output types)

    Args:
        messages: List of messages in Chat Completion API format

    Returns:
        List of messages in Response API format
    """
    converted_messages = []

    for message in messages:
        role = message.get("role")

        # Convert assistant message with tool_calls to function_call
        if role == "assistant" and "tool_calls" in message:
            for tool_call in message["tool_calls"]:
                converted_messages.append({
                    "type": "function_call",
                    "arguments": tool_call["function"]["arguments"],
                    "call_id": tool_call["id"],
                    "name": tool_call["function"]["name"]
                })

        # Convert tool message to function_call_output
        elif role == "tool":
            converted_messages.append({
                "type": "function_call_output",
                "call_id": message["tool_call_id"],
                "output": message["content"]
            })

        # Keep other messages unchanged (system, user, assistant without tool_calls)
        else:
            converted_messages.append(message)

    return converted_messages


original_messages = [
    {"role": "system", "content": "xxx"},
    {
        "content": "I request to know about the pricing of your company?",
        "role": "user",
    },
    {
        "role": "assistant",
        "tool_calls": [
                {
                    "function": {
                        "arguments": '{"request":"What is the pricing of your company?"}',
                        "name": "NmesgxEEXZ-71sx4cqE",
                    },
                    "id": "call_K695KTX4cMVLxeNcIXRVl9qv",
                    "type": "function",
                },
        ],
    },
    {
        "role": "tool",
        "tool_call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
        "content": "Some data was displayed to the user",
    },
    {"role": "user", "content": "What is the pricing of your company?"},
]

response_messages = convert_chat_completion_to_response_format(original_messages)

print(json.dumps(response_messages, indent=2))

[
  {
    "role": "system",
    "content": "xxx"
  },
  {
    "content": "I request to know about the pricing of your company?",
    "role": "user"
  },
  {
    "type": "function_call",
    "arguments": "{\"request\":\"What is the pricing of your company?\"}",
    "call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
    "name": "NmesgxEEXZ-71sx4cqE"
  },
  {
    "type": "function_call_output",
    "call_id": "call_K695KTX4cMVLxeNcIXRVl9qv",
    "output": "Some data was displayed to the user"
  },
  {
    "role": "user",
    "content": "What is the pricing of your company?"
  }
]
